In [1]:


import pandas as pd
import helpers.comparator_functions as cf
import multiprocessing
import dedupe
import math 

from src.helpers.utils import convert_to_tuple_of_tuples, convert_to_year_range
from src.helpers.setup_linking import fields


training_file = "data/training.json"
settings_file = "data/settings"

In [2]:
print("Reading and preparing data", flush=True)

d_mag = pd.read_csv("data/input_mag.csv", keep_default_na=False, na_values=["_"], dtype={"middlename": str})
d_nsf = pd.read_csv("data/input_nsf.csv", keep_default_na=False, na_values=["_"], dtype={"middlename": str})

n_cores = int(multiprocessing.cpu_count() / 2)

for d in [d_nsf, d_mag]:
    for v in ["main_us_institutions_year", "all_us_institutions_year"]:
        d[v] = d.apply(lambda row: convert_to_tuple_of_tuples(row[v]), axis="columns")
    d["year_range"] = d.apply(lambda row: convert_to_year_range(row["year_range"]), axis="columns")

# convert to dict for dedupe 
nsfdata = d_nsf.set_index("grantid_personpos").T.to_dict(orient="dict") # TODO: why is the .T necessary here but not below??
# TODO: also investigate the duplicated columns that python complains about here??
magdata = d_mag.set_index("AuthorId").to_dict(orient="index")


Reading and preparing data


In [4]:
d_nsf.head()
# d_nsf.loc[d_nsf.GrantID == 2034306]

,grantid_personpos,year_range,firstname,lastname,middlename,main_us_institutions_year,all_us_institutions_year
0,2200040_0,"(2022,)",hao,xu,,"((2022, brandeis university),)","((2022, brandeis university),)"
1,2200130_0,"(2022,)",manuel,rivera,l,"((2022, purdue university),)","((2022, purdue university),)"
2,2200390_0,"(2022,)",chandrashekhar,khare,,"((2022, university of california los angeles),)","((2022, university of california los angeles),)"
3,2200426_0,"(2022,)",darayas,patel,n,"((2022, oakwood college),)","((2022, oakwood college),)"
4,2200449_0,"(2022,)",mark,kisin,,"((2022, harvard university),)","((2022, harvard university),)"


In [56]:
for data in [d_nsf, d_mag]:
    data["test"] = data["year_range"]

,grantid_personpos,year_range,firstname,lastname,middlename,main_us_institutions_year,test
0,2200040_0,"(2022,)",hao,xu,,"((2022, brandeis university),)","(2022,)"
1,2200130_0,"(2022,)",manuel,rivera,l,"((2022, purdue university),)","(2022,)"
2,2200390_0,"(2022,)",chandrashekhar,khare,,"((2022, university of california los angeles),)","(2022,)"
3,2200426_0,"(2022,)",darayas,patel,n,"((2022, oakwood college),)","(2022,)"
4,2200449_0,"(2022,)",mark,kisin,,"((2022, harvard university),)","(2022,)"
